In [4]:
import logging
# Imports pour Spark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Import pour les requêtes HTTP
import requests

# Set up the logger
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

spark = SparkSession.builder \
.appName('dpe_existants_rawdata_dl_job') \
.master('spark://spark-master:7077') \
.config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
.config('spark.ui.port', '4041') \
.getOrCreate()

def fetch_from_api():
        url = 'https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-neufs/lines?size=10000&after=50000%2C706146'
        
        all_results = []
        max_calls = 10  # Number of API calls to make
        call_count = 0  # Counter for the number of API calls
        # Loop to handle pagination
        while url and call_count < max_calls:
        # Fetch the JSON data from the URL
            response = requests.get(url)
            
            # Check if the request was successful
            if response.status_code == 200:
                try:
                    # Parse the JSON data
                    data = response.json()
                    
                    # Append the results to the all_results list
                    all_results.extend(data['results'])
                    
                    # Update the URL to the next page
                    url = data.get('next')
                    call_count += 1
                    print(f"Call {call_count} completed. Next URL: {url}")
                    
                    # Check if we have made 10 API calls
                    if call_count >= max_calls:
                        print("Reached maximum number of API calls. Stopping fetch.")
                        break

                except ValueError as e:
                    logger.error(f"Error parsing JSON: {e}")
                    break
            else:
                logger.error(f"Failed to fetch data. Status code: {response.status_code}")
                logger.error(f"Response content: {response.text}")
                break
        
        return all_results

schema = StructType([
    StructField('Conso_chauffage_dépensier_installation_chauffage_n°2', StringType(), True),
    StructField('Volume_stockage_générateur_ECS_n°1', StringType(), True),
    StructField('Conso_é_finale_installation_ECS', StringType(), True),
    StructField('Nom__commune_(BAN)', StringType(), True),
    StructField('Emission_GES_chauffage', StringType(), True),
    StructField('Conso_ECS_é_finale_énergie_n°2', StringType(), True),
    StructField('Conso_ECS_é_finale_énergie_n°1', StringType(), True),
    StructField('Besoin_refroidissement', StringType(), True),
    StructField('Conso_chauffage_dépensier_installation_chauffage_n°1', StringType(), True),
    StructField('Configuration_installation_chauffage_n°2', StringType(), True),
    StructField('Coût_total_5_usages', StringType(), True),
    StructField('Configuration_installation_chauffage_n°1', StringType(), True),
    StructField('Conso_é_finale_dépensier_installation_ECS', StringType(), True),
    StructField('Configuration_installation_ECS', StringType(), True),
    StructField('Type_installation_chauffage_n°2', StringType(), True),
    StructField('Type_installation_chauffage_n°1', StringType(), True),
    StructField('Surface_chauffée_installation_chauffage_n°2', StringType(), True),
    StructField('Surface_chauffée_installation_chauffage_n°1', StringType(), True),
    StructField('Coordonnée_cartographique_X_(BAN)', StringType(), True),
    StructField('Nombre_niveau_logement', StringType(), True),
    StructField('Apports_internes_saison_froid', StringType(), True),
    StructField('Type_installation_ECS_(général)', StringType(), True),
    StructField('Déperditions_murs', StringType(), True),
    StructField('Conso_5_usages_par_m²_é_primaire', StringType(), True),
    StructField('Coût_refroidissement', StringType(), True),
    StructField('Ubat_W/m²_K', StringType(), True),
    StructField('Usage_générateur_ECS_n°1', StringType(), True),
    StructField('Coût_ECS_dépensier', StringType(), True),
    StructField('Emission_GES_auxiliaires', StringType(), True),
    StructField('Année_construction', StringType(), True),
    StructField('Emission_GES_5_usages_par_m²', StringType(), True),
    StructField('Emission_GES_éclairage', StringType(), True),
    StructField('Apports_solaires_saison_froid', StringType(), True),
    StructField('Conso_ECS_dépensier_é_finale', StringType(), True),
    StructField('Adresse_(BAN)', StringType(), True),
    StructField('Date_visite_diagnostiqueur', StringType(), True),
    StructField('N°_étage_appartement', StringType(), True),
    StructField('Type_énergie_générateur_ECS_n°1', StringType(), True),
    StructField('Coût_ECS', StringType(), True),
    StructField('Nombre_niveau_immeuble', StringType(), True),
    StructField('Surface_habitable_desservie_par_installation_ECS', StringType(), True),
    StructField('Surface_habitable_immeuble', StringType(), True),
    StructField("Complément_d'adresse_logement", StringType(), True),
    StructField('Coût_éclairage', StringType(), True),
    StructField('Date_établissement_DPE', StringType(), True),
    StructField('Coût_total_5_usages_énergie_n°2', StringType(), True),
    StructField('Type_générateur_ECS_n°1', StringType(), True),
    StructField('Description_installation_chauffage_n°2', StringType(), True),
    StructField('Description_installation_chauffage_n°1', StringType(), True),
    StructField('N°_voie_(BAN)', StringType(), True),
    StructField('Besoin_ECS', StringType(), True),
    StructField('N°DPE', StringType(), True),
    StructField('Conso_refroidissement_é_finale', StringType(), True),
    StructField('Conso_chauffage_é_primaire', StringType(), True),
    StructField('Appartement_non_visité_(0/1)', StringType(), True),
    StructField('Adresse_brute', StringType(), True),
    StructField('Conso_éclairage_é_primaire', StringType(), True),
    StructField('Qualité_isolation_menuiseries', StringType(), True),
    StructField('Qualité_isolation_murs', StringType(), True),
    StructField('Emission_GES_5_usages_énergie_n°1', StringType(), True),
    StructField('Type_émetteur_installation_chauffage_n°1', StringType(), True),
    StructField('Emission_GES_5_usages_énergie_n°2', StringType(), True),
    StructField('Type_émetteur_installation_chauffage_n°2', StringType(), True),
    StructField('Statut_géocodage', StringType(), True),
    StructField('Emission_GES_ECS_énergie_n°2', StringType(), True),
    StructField('Classe_inertie_bâtiment', StringType(), True),
    StructField('Emission_GES_ECS_énergie_n°1', StringType(), True),
    StructField('Nombre_appartement', StringType(), True),
    StructField('Modèle_DPE', StringType(), True),
    StructField('Description_générateur_chauffage_n°1_installation_n°1', StringType(), True),
    StructField('Description_générateur_ECS_n°1', StringType(), True),
    StructField('Description_générateur_chauffage_n°1_installation_n°2', StringType(), True),
    StructField('Production_électricité_PV_(kWhep/an)', StringType(), True),
    StructField('Conso_5_usages_é_finale', StringType(), True),
    StructField('Nombre_logements_desservis_par_installation_ECS', StringType(), True),
    StructField('N°_département_(BAN)', StringType(), True),
    StructField('Conso_refroidissement_é_primaire', StringType(), True),
    StructField('Méthode_application_DPE', StringType(), True),
    StructField('N°_région_(BAN)', StringType(), True),
    StructField('Surface_habitable_logement', StringType(), True),
    StructField('Code_postal_(brut)', StringType(), True),
    StructField('Deperditions_planchers_bas', StringType(), True),
    StructField('Coordonnée_cartographique_Y_(BAN)', StringType(), True),
    StructField('_rand', StringType(), True),
    StructField('Emission_GES_ECS_dépensier', StringType(), True),
    StructField('Emission_GES_chauffage_énergie_n°2', StringType(), True),
    StructField('Emission_GES_chauffage_énergie_n°1', StringType(), True),
    StructField('Conso_é_finale_dépensier_générateur_ECS_n°1', StringType(), True),
    StructField('Emission_GES_refroidissement', StringType(), True),
    StructField('Classe_altitude', StringType(), True),
    StructField('Description_installation_ECS', StringType(), True),
    StructField('Emission_GES_ECS', StringType(), True),
    StructField('Type_énergie_n°1', StringType(), True),
    StructField('Type_énergie_n°2', StringType(), True),
    StructField('Coût_ECS_énergie_n°2', StringType(), True),
    StructField('Date_réception_DPE', StringType(), True),
    StructField('Coût_ECS_énergie_n°1', StringType(), True),
    StructField('Type_installation_ECS', StringType(), True),
    StructField('Conso_ECS_é_finale', StringType(), True),
    StructField('Emission_GES_5_usages', StringType(), True),
    StructField('Qualité_isolation_plancher_haut_toit_terrase', StringType(), True),
    StructField('Code_postal_(BAN)', StringType(), True),
    StructField('Conso_éclairage_é_finale', StringType(), True),
    StructField('Coût_refroidissement_dépensier', StringType(), True),
    StructField('Date_fin_validité_DPE', StringType(), True),
    StructField('Deperditions_planchers_hauts', StringType(), True),
    StructField('Emission_GES_refroidissement_dépensier', StringType(), True),
    StructField('Type_bâtiment', StringType(), True),
    StructField('Conso_chauffage_générateur_n°1_installation_n°2', StringType(), True),
    StructField('Apports_solaires_saison_chauffe', StringType(), True),
    StructField('Conso_chauffage_générateur_n°1_installation_n°1', StringType(), True),
    StructField('Coût_chauffage', StringType(), True),
    StructField('Déperditions_renouvellement_air', StringType(), True),
    StructField('Conso_chauffage_installation_chauffage_n°2', StringType(), True),
    StructField('Déperditions_portes', StringType(), True),
    StructField('Conso_chauffage_installation_chauffage_n°1', StringType(), True),
    StructField('Conso_ECS_dépensier_é_primaire', StringType(), True),
    StructField('Zone_climatique_', StringType(), True),
    StructField('Conso_refroidissement_dépensier_é_finale', StringType(), True),
    StructField('Usage_générateur_n°1_installation_n°2', StringType(), True),
    StructField('Usage_générateur_n°1_installation_n°1', StringType(), True),
    StructField('Version_DPE', StringType(), True),
    StructField('Deperditions_baies_vitrées', StringType(), True),
    StructField('Conso_chauffage_dépensier_générateur_n°1_installation_n°1', StringType(), True),
    StructField('Type_énergie_générateur_n°1_installation_n°2', StringType(), True),
    StructField('Type_énergie_générateur_n°1_installation_n°1', StringType(), True),
    StructField('Déperditions_ponts_thermiques', StringType(), True),
    StructField('Conso_chauffage_dépensier_générateur_n°1_installation_n°2', StringType(), True),
    StructField('Type_installation_chauffage', StringType(), True),
    StructField('Type_énergie_principale_chauffage', StringType(), True),
    StructField('Qualité_isolation_enveloppe', StringType(), True),
    StructField('Emission_GES_chauffage_dépensier', StringType(), True),
    StructField('Besoin_chauffage', StringType(), True),
    StructField('Conso_ECS_é_primaire', StringType(), True),
    StructField('Etiquette_GES', StringType(), True),
    StructField('Conso_5_usages_é_finale_énergie_n°1', StringType(), True),
    StructField('Conso_5_usages_é_finale_énergie_n°2', StringType(), True),
    StructField("Complément_d'adresse_bâtiment" StringType(), True),
    StructField('Conso_auxiliaires_é_primaire', StringType(), True),
    StructField('Conso_auxiliaires_é_finale', StringType(), True),
    StructField('Conso_é_finale_générateur_ECS_n°1', StringType(), True),
    StructField('Conso_chauffage_é_finale', StringType(), True),
    StructField('Coût_chauffage_dépensier', StringType(), True),
    StructField('Etiquette_DPE', StringType(), True),
    StructField('Conso_refroidissement_dépensier_é_primaire', StringType(), True),
    StructField('Besoin_refroidissement_dépensier', StringType(), True),
    StructField('N°_DPE_immeuble_associé', StringType(), True),
    StructField('Coût_chauffage_énergie_n°1', StringType(), True),
    # StructField('_i', StringType(), True),
    StructField('Coût_chauffage_énergie_n°2', StringType(), True),
    StructField('Qualité_isolation_plancher_bas', StringType(), True),
    StructField('Apports_internes_saison_chauffe_', StringType(), True),
    StructField('Conso_5_usages/m²_é_finale', StringType(), True),
    StructField('Hauteur_sous-plafond', StringType(), True),
    StructField('Identifiant__BAN', StringType(), True),
    StructField('Coût_auxiliaires', StringType(), True),
    StructField('Nom__rue_(BAN)', StringType(), True),
    StructField('Conso_chauffage_dépensier_é_primaire', StringType(), True),
    StructField('Code_INSEE_(BAN)', StringType(), True),
    # StructField('Score_BAN', StringType(), True),
    StructField('Deperditions_enveloppe', StringType(), True),
    StructField('Type_énergie_principale_ECS', StringType(), True),
    StructField('Conso_5_usages_é_primaire', StringType(), True),
    StructField('Conso_chauffage_é_finale_énergie_n°2', StringType(), True),
    StructField('Conso_chauffage_é_finale_énergie_n°1', StringType(), True),
    # StructField('_score', StringType(), True),
    # StructField('_id', StringType(), True),
])


data = fetch_from_api()
print("Number of records fetched:", len(data))
df = spark.createDataFrame(data, schema) 

df = df.withColumn('Année', F.year(F.col('Date_établissement_DPE')))
df_filtered = df.filter(F.col('Année').isin(2021, 2022, 2023))

df_filtered = df_filtered.repartition('Année')
print(df_filtered.count())

df_filtered.write.mode('overwrite').partitionBy('Année').format('parquet') \
    .option("path", "hdfs:///hadoop/dfs/data/DPE/raw_data/dpe_logements_neufs/lot_2/") \
    .saveAsTable("dpe_logements_neufs")


# Stop the SparkSession


SyntaxError: unterminated string literal (detected at line 104) (2596244684.py, line 104)

In [3]:
from pyspark.sql.types import StructType, StructField, StringType
columns = ['Conso_chauffage_dépensier_é_finale', 'Conso_chauffage_dépensier_installation_chauffage_n°2', 'Volume_stockage_générateur_ECS_n°1', 'Conso_é_finale_installation_ECS', 'Nom__commune_(BAN)', 'Emission_GES_chauffage', 'Conso_ECS_é_finale_énergie_n°2', 'Conso_ECS_é_finale_énergie_n°1', 'Besoin_refroidissement', 'Conso_chauffage_dépensier_installation_chauffage_n°1', 'Configuration_installation_chauffage_n°2', 'Coût_total_5_usages', 'Configuration_installation_chauffage_n°1', 'Conso_é_finale_dépensier_installation_ECS', 'Configuration_installation_ECS', 'Type_installation_chauffage_n°2', 'Type_installation_chauffage_n°1', 'Surface_chauffée_installation_chauffage_n°2', 'Surface_chauffée_installation_chauffage_n°1', 'Coordonnée_cartographique_X_(BAN)', 'Nombre_niveau_logement', 'Apports_internes_saison_froid', 'Type_installation_ECS_(général)', 'Déperditions_murs', 'Conso_5_usages_par_m²_é_primaire', 'Coût_refroidissement', 'Ubat_W/m²_K', 'Usage_générateur_ECS_n°1', 'Coût_ECS_dépensier', 'Emission_GES_auxiliaires', 'Année_construction', 'Emission_GES_5_usages_par_m²', 'Emission_GES_éclairage', 'Apports_solaires_saison_froid', 'Conso_ECS_dépensier_é_finale', 'Adresse_(BAN)', 'Date_visite_diagnostiqueur', 'N°_étage_appartement', 'Type_énergie_générateur_ECS_n°1', 'Coût_ECS', 'Nombre_niveau_immeuble', 'Surface_habitable_desservie_par_installation_ECS', 'Surface_habitable_immeuble', "Complément_d'adresse_logement", 'Coût_éclairage', 'Date_établissement_DPE', 'Coût_total_5_usages_énergie_n°2', 'Type_générateur_ECS_n°1', 'Coût_total_5_usages_énergie_n°1', 'Description_installation_chauffage_n°2', 'Description_installation_chauffage_n°1', 'N°_voie_(BAN)', 'Besoin_ECS', 'N°DPE', 'Conso_refroidissement_é_finale', 'Conso_chauffage_é_primaire', 'Appartement_non_visité_(0/1)', 'Adresse_brute', 'Conso_éclairage_é_primaire', 'Qualité_isolation_menuiseries', 'Qualité_isolation_murs', 'Emission_GES_5_usages_énergie_n°1', 'Type_émetteur_installation_chauffage_n°1', 'Emission_GES_5_usages_énergie_n°2', 'Type_émetteur_installation_chauffage_n°2', 'Statut_géocodage', 'Emission_GES_ECS_énergie_n°2', 'Classe_inertie_bâtiment', 'Emission_GES_ECS_énergie_n°1', 'Nombre_appartement', 'Modèle_DPE', 'Description_générateur_chauffage_n°1_installation_n°1', 'Description_générateur_ECS_n°1', 'Description_générateur_chauffage_n°1_installation_n°2', 'Production_électricité_PV_(kWhep/an)', 'Conso_5_usages_é_finale', 'Nombre_logements_desservis_par_installation_ECS', 'N°_département_(BAN)', 'Conso_refroidissement_é_primaire', 'Méthode_application_DPE', 'N°_région_(BAN)', 'Surface_habitable_logement', 'Code_postal_(brut)', 'Deperditions_planchers_bas', 'Coordonnée_cartographique_Y_(BAN)', '_rand', 'Période_construction', 'Emission_GES_ECS_dépensier', 'Emission_GES_chauffage_énergie_n°2', 'Emission_GES_chauffage_énergie_n°1', 'Conso_é_finale_dépensier_générateur_ECS_n°1', 'Emission_GES_refroidissement', 'Classe_altitude', 'Description_installation_ECS', 'Emission_GES_ECS', 'Type_énergie_n°1', 'Type_énergie_n°2', 'Coût_ECS_énergie_n°2', 'Date_réception_DPE', 'Coût_ECS_énergie_n°1', 'Type_installation_ECS', 'Conso_ECS_é_finale', 'Emission_GES_5_usages', 'Qualité_isolation_plancher_haut_toit_terrase', 'Code_postal_(BAN)', 'Conso_éclairage_é_finale', 'Coût_refroidissement_dépensier', 'Date_fin_validité_DPE', 'Deperditions_planchers_hauts', 'Emission_GES_refroidissement_dépensier', 'Type_bâtiment', 'Conso_chauffage_générateur_n°1_installation_n°2', 'Apports_solaires_saison_chauffe', 'Conso_chauffage_générateur_n°1_installation_n°1', 'Coût_chauffage', 'Déperditions_renouvellement_air', 'Conso_chauffage_installation_chauffage_n°2', 'Déperditions_portes', '_geopoint', 'Conso_chauffage_installation_chauffage_n°1', 'Conso_ECS_dépensier_é_primaire', 'Zone_climatique_', 'Conso_refroidissement_dépensier_é_finale', 'Usage_générateur_n°1_installation_n°2', 'Usage_générateur_n°1_installation_n°1', 'Version_DPE', 'Deperditions_baies_vitrées', 'Conso_chauffage_dépensier_générateur_n°1_installation_n°1', 'Type_énergie_générateur_n°1_installation_n°2', 'Type_énergie_générateur_n°1_installation_n°1', 'Déperditions_ponts_thermiques', 'Conso_chauffage_dépensier_générateur_n°1_installation_n°2', 'Type_installation_chauffage', 'Type_énergie_principale_chauffage', 'Qualité_isolation_enveloppe', 'Emission_GES_chauffage_dépensier', 'Besoin_chauffage', 'Conso_ECS_é_primaire', 'Etiquette_GES', 'Conso_5_usages_é_finale_énergie_n°1', 'Conso_5_usages_é_finale_énergie_n°2', "Complément_d'adresse_bâtiment", 'Conso_auxiliaires_é_primaire', 'Conso_auxiliaires_é_finale', 'Conso_é_finale_générateur_ECS_n°1', 'Conso_chauffage_é_finale', 'Coût_chauffage_dépensier', 'Etiquette_DPE', 'Conso_refroidissement_dépensier_é_primaire', 'Besoin_refroidissement_dépensier', 'Type_générateur_n°1_installation_n°2', 'Type_générateur_n°1_installation_n°1', 'N°_DPE_immeuble_associé', 'Coût_chauffage_énergie_n°1', '_i', 'Coût_chauffage_énergie_n°2', 'Qualité_isolation_plancher_bas', 'Apports_internes_saison_chauffe_', 'Conso_5_usages/m²_é_finale', 'Hauteur_sous-plafond', 'Identifiant__BAN', 'Coût_auxiliaires', 'Nom__rue_(BAN)', 'Conso_chauffage_dépensier_é_primaire', 'Code_INSEE_(BAN)', 'Score_BAN', 'Deperditions_enveloppe', 'Type_énergie_principale_ECS', 'Conso_5_usages_é_primaire', 'Conso_chauffage_é_finale_énergie_n°2', 'Conso_chauffage_é_finale_énergie_n°1', '_score', '_id']

def create_schema(columns):
    schema = StructType([
        StructField(column, StringType(), True) 
        for column in columns
    ])
    return schema
schema = create_schema(columns)

# Afficher le schéma
print(schema)


StructType([StructField('Conso_chauffage_dépensier_é_finale', StringType(), True), StructField('Conso_chauffage_dépensier_installation_chauffage_n°2', StringType(), True), StructField('Volume_stockage_générateur_ECS_n°1', StringType(), True), StructField('Conso_é_finale_installation_ECS', StringType(), True), StructField('Nom__commune_(BAN)', StringType(), True), StructField('Emission_GES_chauffage', StringType(), True), StructField('Conso_ECS_é_finale_énergie_n°2', StringType(), True), StructField('Conso_ECS_é_finale_énergie_n°1', StringType(), True), StructField('Besoin_refroidissement', StringType(), True), StructField('Conso_chauffage_dépensier_installation_chauffage_n°1', StringType(), True), StructField('Configuration_installation_chauffage_n°2', StringType(), True), StructField('Coût_total_5_usages', StringType(), True), StructField('Configuration_installation_chauffage_n°1', StringType(), True), StructField('Conso_é_finale_dépensier_installation_ECS', StringType(), True), Struc